In [1]:
from argparse import Namespace
import collections
import re

In [2]:
import pytorch_lightning as pl

In [3]:
from src.data.make_harem import get_example_sets
from src.input.example import InputExample
from src.input.feature import convert_examples_to_features
from src.models.modeling_harem import T5ForHarem

In [ ]:
examples = get_example_sets('../data/harem/', merge_O=True)

In [4]:
hparams = {
    'merge_O': True,
    'labels_mode': 'words',
    'datapath': '../data/harem',
    'remove_accents': False,
    'max_length': 170,
    'stride': 85,
    'target_max_length': 512,
    'pretrained_model': 't5-base-pt/'
}
hparams = Namespace(**hparams)

In [5]:
model = T5ForHarem.from_pretrained(hparams.pretrained_model, hparams=hparams)

In [ ]:
# model.prepare_data()

In [20]:
dl_iter = iter(model.train_dataloader())

In [33]:
batch = next(dl_iter)

In [34]:
target_ids = model.get_target_token_ids(batch)

In [35]:
model.tokenizer.decode(target_ids[0])

'mãe da senhora? R- Minha mãe sempre em casa, cuidando de nós, cuidando de tudo. P - Eram muitos irmãos? R- Eram sete. P - Sete irmãos, homens e mulheres? R- Bom, não. Sete irmãs e dois irmãos, me lembro bem agora. P - São nove. Bastante gente. E lá na cidade, a senhora estudou na cidade mesmo, lá em [Outro] Conchas [Local]? R- Estudava em [Outro] Fartura [Local]. Não em [Outro] Conchas [Local] foi só mamãe. P - Em [Outro] Fartura [Local]. E qual escola era? R- Só no [Outro] Grupo Escolar de Fartura [Organização]. P - Ficou lá até que época? R- Até [Outro] 50 [Tempo]. P - Casou lá? E Quem era seu marido? Casei lá. [Outro]'

In [36]:
model.tokenizer.decode(batch[0][0])

'Reconhecer Entidade: mãe da senhora? R- Minha mãe sempre em casa, cuidando de nós, cuidando de tudo. P - Eram muitos irmãos? R- Eram sete. P - Sete irmãos, homens e mulheres? R- Bom, não. Sete irmãs e dois irmãos, me lembro bem agora. P - São nove. Bastante gente. E lá na cidade, a senhora estudou na cidade mesmo, lá em Conchas? R- Estudava em Fartura. Não em Conchas foi só mamãe. P - Em Fartura. E qual escola era? R- Só no Grupo Escolar de Fartura. P - Ficou lá até que época? R- Até 50. P - Casou lá? E Quem era seu marido? Casei lá.'

In [6]:
examples = model.get_examples()

In [7]:
example = examples['valid'][3]

In [8]:
class InputSpanFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 unique_id,
                 doc_span_index,
                 tokens,
                 token_to_orig_map,
                 token_is_max_context,
                 input_ids,
                 attention_mask,
                 label_tags,
                 target_ids):
        self.unique_id = unique_id
        self.doc_span_index = doc_span_index
        self.tokens = tokens
        self.token_to_orig_map = token_to_orig_map
        self.token_is_max_context = token_is_max_context
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.label_tags = label_tags
        self.target_ids = target_ids
        
    def start_ignore_index(self, value: int = -100) -> int:
        index = len(self.target_ids)
        if value in self.target_ids:
            index = self.target_ids.index(value)
        return index

    @property
    def clean_target_ids(self,):
        return self.target_ids[:self.start_ignore_index()]

In [9]:
def _check_is_max_context(doc_spans, cur_span_index, position):
    """Check if this is the 'max context' doc span for the token."""

    # Because of the sliding window approach taken to scoring documents, a single
    # token can appear in multiple documents. E.g.
    #  Doc: the man went to the store and bought a gallon of milk
    #  Span A: the man went to the
    #  Span B: to the store and bought
    #  Span C: and bought a gallon of
    #  ...
    #
    # Now the word 'bought' will have two scores from spans B and C. We only
    # want to consider the score with "maximum context", which we define as
    # the *minimum* of its left and right context (the *sum* of left and
    # right context will always be the same, of course).
    #
    # In the example the maximum context for 'bought' would be span C since
    # it has 1 left context and 3 right context, while span B has 4 left context
    # and 0 right context.
    best_score = None
    best_span_index = None
    for (span_index, doc_span) in enumerate(doc_spans):
        end = doc_span.start + doc_span.length - 1
        if position < doc_span.start:
            continue
        if position > end:
            continue
        num_left_context = position - doc_span.start
        num_right_context = end - position
        score = min(num_left_context, num_right_context) + \
            0.01 * doc_span.length
        if best_score is None or score > best_score:
            best_score = score
            best_span_index = span_index

    return cur_span_index == best_span_index

In [10]:
def convert_example_to_spanfeatures(example, max_seq_length, tokenizer, doc_stride, max_target_length=512):
    """Loads a data file into a list of `InputBatch`s."""
    '''I still need to fix the example'''

    prefix = 'Reconhecer Entidade:'
    prefix_tokens = tokenizer.tokenize(prefix)
    
    features = []

    tok_to_orig_index = []
    orig_to_tok_index = []
    all_doc_tokens = []
    all_doc_labels = []
    all_doc_labels_mask = []
    doc_text = example.source_words
    doc_labels = example.word_labels
    assert len(doc_labels) == len(doc_text)

    for (i, token) in enumerate(doc_text):
        orig_to_tok_index.append(len(all_doc_tokens))
        sub_tokens = tokenizer.tokenize(token)
        token_label = doc_labels[i]
        for j, sub_token in enumerate(sub_tokens):
            tok_to_orig_index.append(i)
            all_doc_tokens.append(sub_token)
            if j > 0 and token_label != 'O':
                _label = f'I-{token_label.split("-")[-1]}'
            else:
                _label = token_label
            all_doc_labels.append(_label)


    # The -1 - len(prefix_tokens) accounts for EOS and prefix
    max_tokens_for_doc = max_seq_length - 1 - len(prefix_tokens)

    # We can have documents that are longer than the maximum sequence length.
    # To deal with this we do a sliding window approach, where we take chunks
    # of the up to our max length with a stride of `doc_stride`.
    _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
        "DocSpan", ["start", "length"])
    doc_spans = []
    start_offset = 0
    while start_offset < len(all_doc_tokens):
        length = len(all_doc_tokens) - start_offset
        if length > max_tokens_for_doc:
            length = max_tokens_for_doc
        doc_spans.append(_DocSpan(start=start_offset, length=length))
        if start_offset + length == len(all_doc_tokens):
            break
        start_offset += min(length, doc_stride)

    unique_count = 0
    for (doc_span_index, doc_span) in enumerate(doc_spans):
        tokens = []
        label_tags = []
        token_to_orig_map = {}
        token_is_max_context = {}

        found_start = False
        for i in range(doc_span.length):
            split_token_index = doc_span.start + i
            
            if not found_start:
                t = all_doc_tokens[split_token_index]
                found_start = t.startswith('▁')
            
            if found_start:
                token_to_orig_map[len(
                    tokens)] = tok_to_orig_index[split_token_index]

                is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                                       split_token_index)
                token_is_max_context[len(tokens)] = is_max_context
                tokens.append(all_doc_tokens[split_token_index])
                label_tags.append(all_doc_labels[split_token_index])

        if tokens[-1] == '▁':
            tokens = tokens[:-1]
            label_tags = label_tags[:-1]
            del token_is_max_context[len(tokens)]
            
        retokenized = tokenizer.tokenize(tokenizer.convert_tokens_to_string(tokens))
        assert len(retokenized) == len(tokens), f'{len(retokenized)} - {len(tokens)} / {retokenized} - {tokens}'
        assert len(tokens) == len(label_tags) == len(token_is_max_context)
        
        tokens = prefix_tokens + tokens
        tokens.append(tokenizer.eos_token)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        attention_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            attention_mask.append(0)

        assert len(input_ids) == max_seq_length
        assert len(attention_mask) == max_seq_length
        
        
        
        idx = 0
        target_ids = []
        while idx < len(label_tags):

            label = label_tags[idx]

            if label == 'O':
                j = idx + 1
                while j < len(label_tags) and label_tags[j] == 'O':
                    j += 1
                # adds the span
                _ids = input_ids[len(prefix_tokens) + idx: len(prefix_tokens) + j]
                entity = labels2words.get(label, f'<{label}>')
                enitity_ids = tokenizer.encode(entity)
                
                target_ids += _ids + enitity_ids
                idx = j
            else:
                j = idx + 1
                ent_label = label.split('-')[-1]
                while j < len(label_tags) and label_tags[j] == f'I-{ent_label}':
                    j += 1
                # adds the span
                _ids = input_ids[len(prefix_tokens) + idx: len(prefix_tokens) + j]        
                entity = labels2words.get(ent_label, f'<{ent_label}>')
                enitity_ids = tokenizer.encode(entity)
                
                target_ids += _ids + enitity_ids
                idx = j
        
        target_ids += [tokenizer.eos_token_id]
        
        assert len(target_ids) < max_target_length, f'{len(target_ids)}'
        
        while len(target_ids) < max_target_length:
            target_ids.append(-100) # to ignore on loss

        feature = InputSpanFeatures(
            unique_id=f'{unique_count}',
            doc_span_index=doc_span_index,
            tokens=tokens,
            token_to_orig_map=token_to_orig_map,
            token_is_max_context=token_is_max_context,
            input_ids=input_ids,
            attention_mask=attention_mask,
            label_tags=label_tags,
            target_ids=target_ids
            
        )

        unique_count += 1

        features.append(feature)

    return features

In [11]:
labels2words = model.labels2words

In [12]:
entities = list(labels2words.values())

In [13]:
features = convert_example_to_spanfeatures(example, 170, model.tokenizer, 85)

In [14]:
feat = features[3]

In [15]:
input_ids = feat.input_ids
target_ids = feat.target_ids

In [16]:
input_tokens = model.tokenizer.convert_ids_to_tokens(input_ids[7:])

In [17]:
target_string = model.tokenizer.decode(feat.clean_target_ids)
for ent in entities:
    target_string = target_string.replace(ent, '')
target_string = re.sub(' {2,}', ' ', target_string)

In [18]:
target_tokens = model.tokenizer.tokenize(target_string)

In [19]:
all([it == tt for it, tt in zip(input_tokens, target_tokens)])

True

In [22]:
for j, example in enumerate(examples['train']):
    features = convert_example_to_spanfeatures(example, 170, model.tokenizer, 85)
    
    for feat in features:
        input_ids = feat.input_ids
        target_ids = feat.target_ids

        input_tokens = model.tokenizer.convert_ids_to_tokens(input_ids[7:])

        target_string = model.tokenizer.decode(feat.clean_target_ids)
        for ent in entities:
            target_string = target_string.replace(ent, '')
#         target_string = re.sub(' {2,}', ' ', target_string)

        target_tokens = model.tokenizer.tokenize(target_string)

        assert all([it == tt for it, tt in zip(input_tokens, target_tokens)])

AssertionError: 

In [23]:
for it, tt in zip(input_tokens, target_tokens):
    print(it, tt)

▁página ▁página
▁de ▁de
▁O ▁O
pin pin
ião ião
▁e ▁e
▁di ▁di
ga ga
- -
nos nos
▁o ▁o
▁que ▁que
▁pensa ▁pensa
▁sobre ▁sobre
▁este ▁este
▁conflito ▁conflito
. .
▁Formação ▁Formação
▁Profissional ▁Profissional
▁é ▁é
▁a ▁a
▁melhor ▁melhor
▁opção ▁opção
▁Os ▁Os
▁alunos ▁alunos
▁com ▁com
▁o ▁o
▁12 ▁12
o o
▁ano ▁ano
▁que ▁que
▁não ▁não
▁que ▁que
rem rem
▁seguir ▁seguir
▁o ▁o
▁Ensino ▁Ensino
▁Superior ▁Superior
▁devem ▁devem
▁aposta ▁aposta
r r
▁na ▁na
▁formação ▁formação
▁profissional ▁profissional
▁antes ▁antes
▁de ▁de
▁começar ▁começar
em em
▁a ▁a
▁trabalhar ▁trabalhar
. .
▁A ▁A
▁mensagem ▁mensagem
▁foi ▁foi
▁deixada ▁deixada
▁pela ▁pela
▁orientado ▁orientado
ra ra
▁do ▁do
▁ ▁
work work
s s
hop hop
▁de ▁de
▁Técnica ▁Técnica
s s
▁de ▁de
▁Pro ▁Pro
cura cura
▁de ▁de
▁Em ▁Em
pre pre
go go
, ,
▁terça ▁terça
- -
feira feira
, ,
▁em ▁em
▁Ma ▁Ma
fra fra
▁ ▁
. .
▁9 ▁9
-4 -4
-20 -20
03 03
▁18 ▁18
: :
36 36
▁0 ▁0
▁Com ▁Com
en en
tários tários
▁ ?
? ▁Paz
▁Paz ▁sim
▁sim ,
, ▁Guerra
▁Guerra ▁não
▁não !
! 

In [29]:
j

10

In [125]:
feat = features[3]

In [24]:
model.tokenizer.decode(feat.input_ids)

'Reconhecer Entidade: página de Opinião e diga-nos o que pensa sobre este conflito. Formação Profissional é a melhor opção Os alunos com o 12o ano que não querem seguir o Ensino Superior devem apostar na formação profissional antes de começarem a trabalhar. A mensagem foi deixada pela orientadora do workshop de Técnicas de Procura de Emprego, terça-feira, em Mafra. 9-4-2003 18:36 0 Comentários? Paz sim, Guerra não!? Os alunos da Escola Secundária José Saramago juntaram-se contra a Guerra e a violência. No 21o dia de confrontos no Iraque, alunos e professores percorreram algumas ruas da vila de Mafra, distribuíram folhetos e soltaram pombas brancas a favor da Paz'

In [25]:
model.tokenizer.decode(feat.clean_target_ids)

'página de Opinião e diga-nos o que pensa sobre este conflito. Formação Profissional é a melhor opção Os alunos com o [Outro] 12o [Valor] ano que não querem seguir o Ensino Superior devem apostar na formação profissional antes de começarem a trabalhar. A mensagem foi deixada pela orientadora do workshop de Técnicas de Procura de Emprego, terça-feira, em [Outro] Mafra [Local]. [Outro] 9-4-2003 [Tempo] 18:36 [Tempo] 0 [Valor] Comentários? Paz sim, Guerra não!? Os alunos da [Outro] Escola Secundária José Saramago [Organização] juntaram-se contra a Guerra e a violência. No [Outro] 21o [Valor] dia de confrontos no [Outro] Iraque [Local], alunos e professores percorreram algumas ruas da vila de [Outro] Mafra [Local], distribuíram folhetos e soltaram pombas brancas a favor da Paz [Outro]'

In [111]:
len(feat.target_ids)

512

In [93]:
len(feat.input_ids)

300

In [95]:
feat.target_ids[-3:]

[14133, 254, 1035]

In [21]:
span_examples = []

for example in examples['train']:
    target = example.target
    target_tokens = model.tokenizer.tokenize(target)
    n_spans = len(target_tokens) // stride

    for i in range(n_spans):
        start = stride * i
        end = min(stride * i + max_length, len(target_tokens))

        span_tokens = target_tokens[start:end]
        target_span_words = model.tokenizer.convert_tokens_to_string(
            span_tokens).split(' ')
        last_ent_index = [i for i, w in enumerate(
            target_span_words) if w in possible_endings]
        
        if len(last_ent_index):
            last_ent_index = last_ent_index[-1]
            target_span_words = target_span_words[:last_ent_index+1]
        else:
            target_span_words = ['[Outro]']
            
        target_span_words = [
            w for w in target_span_words if w in example.target_words]
        source_span_words = [
            w for w in target_span_words if w in example.source_words]

        span_examples.append(InputExample(
            source_span_words, target_span_words))

In [14]:
target = example.target
target_tokens = model.tokenizer.tokenize(target)

n_spans = len(target_tokens) // stride
n_spans

9

In [15]:
span_examples = []

for i in range(n_spans):
    start = stride * i
    end = min(stride * i + max_length, len(target_tokens))
    
    span_tokens = target_tokens[start:end]
    target_span_words = model.tokenizer.convert_tokens_to_string(span_tokens).split(' ')
    last_ent_index = [i for i, w in enumerate(target_span_words) if w in possible_endings][-1]
    target_span_words = target_span_words[:last_ent_index+1]
    target_span_words = [w for w in target_span_words if w in example.target_words]
    source_span_words = [w for w in target_span_words if w in example.source_words]
    
    span_examples.append(InputExample(source_span_words, target_span_words))

In [16]:
features = convert_examples_to_features(span_examples, model.tokenizer, max_length=256, prefix='Reconhecer Entidades:')

In [19]:
model.tokenizer.decode(features[1].source_token_ids)

'Reconhecer Entidades:. O seu pai, Henru James, era um teologo seguidor de Emanuel Swedenborg. Um dos seus irmaos foi o conhecido novelista Henry James. Concluiu os seus estudos de medicina, em 1870, na Universidade de Harvard, onde iniciou a sua carreira como professor de fisiologia em 1872. A partir de 1880 ensinou psicologia e filosofia em Harvard'

In [20]:
model.tokenizer.decode(features[1].target_ids)

'. O seu pai, [Outro] Henru James [Pessoa], era um teologo seguidor de [Outro] Emanuel Swedenborg [Pessoa]. Um dos seus irmaos foi o conhecido novelista [Outro] Henry James [Pessoa]. Concluiu os seus estudos de medicina, em [Outro] 1870 [Tempo], na [Outro] Universidade de Harvard [Organizaç ⁇ o], onde iniciou a sua carreira como professor de fisiologia em [Outro] 1872 [Tempo]. A partir de [Outro] 1880 [Tempo] ensinou psicologia e filosofia em [Outro] Harvard [Organizaç ⁇ o]'

# From model

In [7]:
model.prepare_data()

NameError: name 'collections' is not defined

In [15]:
len(model.train_dataset)

869

In [8]:
dl_test = model.test_dataloader()

In [9]:
batch = next(iter(dl_test))

In [10]:
target_ids = model.get_target_token_ids(batch)

In [15]:
model.tokenizer.decode(target_ids[0])

'W. JAMES [Pessoa] Willian James [Pessoa] Willian James [Pessoa], filosofo e psicologo. Foi o mais influente dos pensadores dos [Outro] EUA [Local], criador do pragmatismo. Nasceu e, [Outro] Nova Iorque [Local], a [Outro] 11 de Janeiro de 1842 [Tempo]. O seu pai, [Outro] Henru James [Pessoa], era um teologo seguidor de [Outro] Emanuel Swedenborg [Pessoa]. Um dos seus irmaos foi o conhecido novelista [Outro] Henry James [Pessoa]. Concluiu os seus estudos de medicina, em [Outro] 1870 [Tempo], na [Outro] Universidade de Harvard [Organizaç ⁇ o], onde iniciou a sua carreira como professor de fisiologia em [Outro] 1872 [Tempo]. A partir de [Outro] 1880 [Tempo] ensinou psicologia e filosofia em [Outro] Harvard [Organizaç ⁇ o], universidade que abandonou em [Outro] 1907 [Tempo], proferindo conferencias nas universidades de [Outro] Columbia [Organizaç ⁇ o] e [Outro] Oxford [Organizaç ⁇ o]. Morreu em [Outro] Chocorua [Local], [Outro] New Hampshire [Local], a [Outro] 26 de Agosto de 1910 [Tempo]'